In [ ]:
import numpy as np
import pickle

In [ ]:
filenames = [
    'Gummy Worm Dataset__SVM__Gummy Worm Dataset Family__AbsoluteValues__20250524_140318.pkl',
    'Gummy Worm Dataset__Neural Network__Gummy Worm Dataset Family__AbsoluteValues__20250524_140318.pkl',
    'Gummy Worm Dataset__Logistic Regression__Gummy Worm Dataset Family__AbsoluteValues__20250524_140318.pkl',
    'Gummy Worm Dataset__Random Forest__Gummy Worm Dataset Family__AbsoluteValues__20250524_140318.pkl',
    'Exclamation Mark Dataset__SVM__Exclamation Mark Dataset Family__AbsoluteValues__20250529_134349.pkl',
    'Exclamation Mark Dataset__Neural Network__Exclamation Mark Dataset Family__AbsoluteValues__20250529_134349.pkl',
    'Exclamation Mark Dataset__Logistic Regression__Exclamation Mark Dataset Family__AbsoluteValues__20250529_134349.pkl',
    'Exclamation Mark Dataset__Random Forest__Exclamation Mark Dataset Family__AbsoluteValues__20250529_134349.pkl'
]

In [3]:
eces = []
aces = []
fces = []
for filename in filenames:
    with (open('../varying_bins_dataset_family/data/' + filename, 'rb') as file):
        print(filename)
        results = pickle.load(file)

        means = results["Means"]
        eces.append(means["ECE"])
        aces.append(means["ACE"])
        fces.append(means["FCE"])
        
eces = np.array(eces)
aces = np.array(aces)
fces = np.array(fces)

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x00000234E7129F50>>
Traceback (most recent call last):
  File "C:\Users\Moritz\PycharmProjects\bachelor_thesis\.venv\Lib\site-packages\ipykernel\ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


Gummy Worm Dataset__Logistic Regression__Gummy Worm Dataset Family__AbsoluteValues__20250524_140318.pkl
Gummy Worm Dataset__Random Forest__Gummy Worm Dataset Family__AbsoluteValues__20250524_140318.pkl
Exclamation Mark Dataset__SVM__Exclamation Mark Dataset Family__AbsoluteValues__20250529_134349.pkl
Exclamation Mark Dataset__Neural Network__Exclamation Mark Dataset Family__AbsoluteValues__20250529_134349.pkl
Exclamation Mark Dataset__Logistic Regression__Exclamation Mark Dataset Family__AbsoluteValues__20250529_134349.pkl
Exclamation Mark Dataset__Random Forest__Exclamation Mark Dataset Family__AbsoluteValues__20250529_134349.pkl


In [4]:
eces_stds = np.std(eces, axis=-1)
aces_stds = np.std(aces, axis=-1)
fces_stds = np.std(fces, axis=-1)

print(eces_stds.shape)

(8,)


In [5]:
experiments = ["SVM - GW", "NN - GW", "LR - GW", "RF - GW", "SVM - EX", "NN - EX", "LR - EX", "RF - EX"]

df_list = []

In [6]:
for i, name in enumerate(experiments):
    df_list.append({
        "experiment": name,
        "ECE": eces_stds[i],
        "FCE": fces_stds[i],
        "ACE": aces_stds[i]
    })

In [7]:
import pandas as pd

pd.set_option('display.max_columns', None)
df = pd.DataFrame(df_list)
print(df)

  experiment       ECE       FCE       ACE
0   SVM - GW  0.001559  0.001340  0.001144
1    NN - GW  0.000799  0.000812  0.000680
2    LR - GW  0.011352  0.011730  0.005492
3    RF - GW  0.000964  0.001518  0.000417
4   SVM - EX  0.005920  0.005672  0.003791
5    NN - EX  0.002863  0.002627  0.004192
6    LR - EX  0.003073  0.002799  0.004681
7    RF - EX  0.000338  0.000589  0.000359


In [8]:
### Print Latex code for table

model_names = df["experiment"]
data_only = df.drop(columns=["experiment"])

def highlight_row(row):
    ranks = row.rank(method="min", ascending=False)
    formatted = row.astype(str)

    for col in row.index:
        val = row[col]
        if ranks[col] == 1:
            formatted[col] = f"\\textbf{{{val:.5f}}}"
        elif ranks[col] == 2:
            formatted[col] = f"\\underline{{{val:.5f}}}"
        else:
            formatted[col] = f"{val:.5f}"
    return formatted

# Apply formatting row-wise
styled_data = data_only.apply(highlight_row, axis=1)

# Add back the model column
styled_df = pd.concat([model_names, styled_data], axis=1)

latex_code = styled_df.to_latex(index=False, caption="Sensitivity to \textit{bins} hyperparameter of various binned metrics expressed in standard deviation of metric values between 1 and 100 bins. Bold and underlined values indicate most sensitive and second most sensitive values, respectively. GWF, EXF, NN, LR, RF are abbreviations for Gummy Worm Family, Exclamation Mark Family, Neural Network, Logistic Regression and Random Forest, respectively")
print(latex_code)

\begin{table}
\caption{Mean absolute error between True ECE$_{dists}^{15}$ and various metrics. Bold and underlined values indicate top-1 and top-2 values, respectively. GW, EX, NN, LR, RF are abbreviations for Gummy Worm, Exclamation Mark, Neural Network, Logistic Regression and Random Forest, respectively.}
\begin{tabular}{llll}
\toprule
experiment & ECE & FCE & ACE \\
\midrule
SVM - GW & \textbf{0.00156} & \underline{0.00134} & 0.00114 \\
NN - GW & \underline{0.00080} & \textbf{0.00081} & 0.00068 \\
LR - GW & \underline{0.01135} & \textbf{0.01173} & 0.00549 \\
RF - GW & \underline{0.00096} & \textbf{0.00152} & 0.00042 \\
SVM - EX & \textbf{0.00592} & \underline{0.00567} & 0.00379 \\
NN - EX & \underline{0.00286} & 0.00263 & \textbf{0.00419} \\
LR - EX & \underline{0.00307} & 0.00280 & \textbf{0.00468} \\
RF - EX & 0.00034 & \textbf{0.00059} & \underline{0.00036} \\
\bottomrule
\end{tabular}
\end{table}
